In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Intro_to_image_processing/Final_Project')
!pwd
!ls

/content/drive/MyDrive/Colab Notebooks/Intro_to_image_processing/Final_Project
AMP.ipynb  DAT	DAT.ipynb  dat_model.py  DAT_x4.pth  DRCT.ipynb  HAT  images


In [ ]:
!pip install torch torchvision
!pip install addict lmdb tb-nightly


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# Clone the DAT repository
!git clone https://github.com/XPixelGroup/HAT.git


fatal: destination path 'HAT' already exists and is not an empty directory.


In [ ]:
# Change directory to the cloned repo
%cd HAT

# View the contents of requirements.txt
!cat requirements.txt



/content/drive/MyDrive/Colab Notebooks/Intro_to_image_processing/Final_Project/HAT
einops
torch>=1.7
basicsr==1.3.4.9


In [ ]:
!pip install -r requirements.txt --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 836.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for basicsr: filename=basicsr-1.3.4.9-py3-none-any.whl size=194421 sha256=74201c1efb06f258c15f7869c828093f705bf4075fcc46606de6565c6f75729d
  Stored in directory: /root/.cache/pip/wheels/37/19/97/8206d928857f2ef0625cb621b689005610515b4841be4e5ddb
Successfully built basicsr


In [ ]:
!pwd
!ls

/content/drive/MyDrive/Colab Notebooks/Intro_to_image_processing/Final_Project/HAT
cog.yaml  experiments  hat	  LICENSE  predict.py  requirements.txt  setup.cfg  VERSION
datasets  figures      input_dir  options  README.md   results		 setup.py


In [ ]:
def adaptive_weighted_mean_filter(image, kernel_size=9, max_intensity_diff=20):
    """
    Apply Adaptive Weighted Mean Filter to the input image.

    Parameters:
        image (numpy.ndarray): Input image (color or grayscale).
        kernel_size (int): Size of the kernel (odd integer).
        max_intensity_diff (int): Maximum intensity difference to consider when calculating weights.

    Returns:
        numpy.ndarray: Filtered image.
    """
    filtered_image = np.zeros_like(image)

    pad = kernel_size // 2
    padded_image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REFLECT)

    for i in range(pad, padded_image.shape[0] - pad):
        for j in range(pad, padded_image.shape[1] - pad):
            neighborhood = padded_image[i-pad:i+pad+1, j-pad:j+pad+1]
            center_pixel = padded_image[i, j]
            weights = np.exp(-np.abs(neighborhood - center_pixel) / max_intensity_diff)
            normalized_weights = weights / np.sum(weights, axis=(0, 1))  # Normalize weights for each channel
            filtered_value = np.sum(normalized_weights * neighborhood, axis=(0, 1))
            filtered_image[i-pad, j-pad] = np.clip(filtered_value, 0, 255)

    return filtered_image.astype(np.uint8)


In [ ]:
import os
import cv2
import numpy as np

def max_filter(image, kernel_size):
    return cv2.dilate(image, np.ones((kernel_size, kernel_size), np.uint8))

def min_filter(image, kernel_size):
    return cv2.erode(image, np.ones((kernel_size, kernel_size), np.uint8))

def average_filter(image, kernel_size):
    return cv2.blur(image, (kernel_size, kernel_size))

def median_filter(image, kernel_size):
    return cv2.medianBlur(image, kernel_size)

def adaptive_bilateral_filter(image, d, sigma_color, sigma_space):
    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

def process_images_in_folder(input_folder, output_folder, kernel_size):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    compare_folder="compare"
    if not os.path.exists(compare_folder):
        os.makedirs(compare_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
            compare_path = os.path.join(compare_folder, filename)

            original_image = cv2.imread(input_path)
            if original_image is None:
                print(f"Could not read image: {input_path}")
                continue


            d = 9 # Diameter of each pixel neighborhood
            sigma_color = 20  # Filter sigma in the color space
            sigma_space = 200  # Filter sigma in the coordinate space
            filtered_image = adaptive_bilateral_filter(original_image, d, sigma_color, sigma_space)

            cv2.imwrite(output_path, filtered_image)
            comparison = np.concatenate((original_image, filtered_image), axis=1)
            cv2.imwrite(compare_path, comparison)
            print(f"Processed: {input_path} -> {output_path}")

if __name__ == "__main__":
    input_folder = "results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom"  # Specify the folder containing your input images
    output_folder = "filtered_image"  # Specify the folder where you want to save the filtered images
    kernel_size = 3  # Adjust kernel size as needed

    process_images_in_folder(input_folder, output_folder, kernel_size)


Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/1_HAT_GAN_Real_SRx4.png -> filtered_image/1_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/10_HAT_GAN_Real_SRx4.png -> filtered_image/10_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/11_HAT_GAN_Real_SRx4.png -> filtered_image/11_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/12_HAT_GAN_Real_SRx4.png -> filtered_image/12_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/13_HAT_GAN_Real_SRx4.png -> filtered_image/13_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/14_HAT_GAN_Real_SRx4.png -> filtered_image/14_HAT_GAN_Real_SRx4.png
Processed: results/HAT_GAN_Real_SRx4_archived_20240531_024714/visualization/custom/15_HAT_GAN_Re

In [ ]:
!python hat/test.py -opt options/test/HAT_GAN_Real_SRx4.yml

Disable distributed.
Path already exists. Rename it to /content/drive/MyDrive/Colab Notebooks/Intro_to_image_processing/Final_Project/HAT/results/HAT_GAN_Real_SRx4_archived_20240531_040340
2024-05-31 04:03:40,718 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.3.4.9
	PyTorch: 2.3.0+cu121
	TorchVision: 0.18.0+cu121
2024-05-31 04:03:40,719 INFO: 
  name: HAT_GAN_Real_SRx4
  model_type: HATModel
  scal